In [3]:
import pprint
import config
import biothings
biothings.config_for_app(config)
import biothings.utils.mongo as mongo
col = mongo.get_src_db()["entrez_go"]
print(col)
docs = [d for d in col.find()]
print("%d GO documents" % len(docs))


Collection(Database(MongoClient(host=['su05:27017'], document_class=dict, tz_aware=False, connect=True), 'genedoc_src'), 'entrez_go')
210030 GO documents


# Structure analysis

In [6]:
import biothings.utils.inspect as inspect
import importlib
importlib.reload(inspect)
mapt = inspect.inspect_docs(docs,mode="type")
pprint.pprint(mapt)

{'_id': {<class 'str'>: {}},
 'go': {'BP': {'evidence': {<class 'str'>: {}},
               'id': {<class 'str'>: {}},
               'pubmed': {<class 'list'>: {<class 'int'>: {}},
                          <class 'int'>: {}},
               'qualifier': {<class 'str'>: {}},
               'term': {<class 'str'>: {}},
               <class 'list'>: {'evidence': {<class 'str'>: {}},
                                'id': {<class 'str'>: {}},
                                'pubmed': {<class 'list'>: {<class 'int'>: {}},
                                           <class 'int'>: {}},
                                'qualifier': {<class 'str'>: {}},
                                'term': {<class 'str'>: {}}}},
        'CC': {'evidence': {<class 'str'>: {}},
               'id': {<class 'str'>: {}},
               'pubmed': {<class 'list'>: {<class 'int'>: {}},
                          <class 'int'>: {}},
               'qualifier': {<class 'str'>: {}},
               'term': {<class 'str

# ElasticSearch mapping generation

In [9]:
import importlib
importlib.reload(inspect)
mapm = inspect.inspect_docs(docs,mode="mapping")
pprint.pprint(mapm) # same mode=type but some extra information about how values look like, relevant to ES

{'_id': {<class 'str'>: {}},
 'go': {'BP': {<class 'list'>: {'evidence': {<class 'str'>: {}},
                                'id': {<class 'str'>: {}},
                                'pubmed': {<class 'list'>: {<class 'int'>: {}}},
                                'qualifier': {<class 'str'>: {}},
                                'term': {<class 'str'>: {'split': {}}}}},
        'CC': {<class 'list'>: {'evidence': {<class 'str'>: {}},
                                'id': {<class 'str'>: {}},
                                'pubmed': {<class 'list'>: {<class 'int'>: {}}},
                                'qualifier': {<class 'str'>: {'split': {}}},
                                'term': {<class 'str'>: {'split': {}}}}},
        'MF': {<class 'list'>: {'evidence': {<class 'str'>: {}},
                                'id': {<class 'str'>: {}},
                                'pubmed': {<class 'list'>: {<class 'int'>: {}}},
                                'qualifier': {<class 'str'>: {'sp

In [10]:
import biothings.utils.es as es
mapping = es.generate_es_mapping(mapm)
pprint.pprint(mapping)

{'go': {'properties': {'BP': {'properties': {'evidence': {'analyzer': 'string_lowercase',
                                                          'type': 'string'},
                                             'id': {'analyzer': 'string_lowercase',
                                                    'type': 'string'},
                                             'pubmed': {'type': 'integer'},
                                             'qualifier': {'analyzer': 'string_lowercase',
                                                           'type': 'string'},
                                             'term': {'type': 'string'}}},
                       'CC': {'properties': {'evidence': {'analyzer': 'string_lowercase',
                                                          'type': 'string'},
                                             'id': {'analyzer': 'string_lowercase',
                                                    'type': 'string'},
                                    

# Structure statistics

In [19]:
import biothings.utils.inspect as inspect
import importlib
importlib.reload(inspect)
zedocs = docs#[:1000]
#mapt = inspect.inspect_docs(zedocs,mode="stats",merge=True)
#inspect.merge_scalar_list(mapt,"stats")
#print("BP contains scalar")
#pprint.pprint(mapt["go"]["BP"]["pubmed"])
print("BP")
pprint.pprint(mapt)
bp = mapt["go"]["BP"]
pm_aslist = 0
pm_aslist_in_bp_asscalar = 0
pm_aslist_in_bp_as_list = 0
pm_asscalar = 0
pm_asscalar_in_bp_asscalar = 0
pm_asscalar_in_bp_aslist = 0
pm_sumextend = 0
for d in zedocs:
    if type(d["go"].get("BP",{})) == list:
        for bp in d["go"].get("BP"):
            if type(bp.get("pubmed",{})) == list:
                pm_aslist += 1
                pm_aslist_in_bp_as_list += 1
                pm_sumextend += len(bp)
                break
            elif bp.get("pubmed",{}):
                pm_asscalar += 1
                pm_asscalar_in_bp_aslist += 1
                pm_sumextend += len(d["go"].get("BP"))
                break
    elif d["go"].get("BP"):
        bp = d["go"].get("BP")
        if type(bp.get("pubmed",{})) == list:
            pm_aslist_in_bp_asscalar += 1
            pm_aslist += 1
            pm_sumextend += len(bp)
        elif bp.get("pubmed"):
            pm_asscalar += 1
            pm_asscalar_in_bp_asscalar += 1
            pm_sumextend += 1
print("pm_aslist: %s" % pm_aslist)
print("pm_asscalar: %s" % pm_asscalar)
print("pm_aslist_in_bp_asscalar: %s" % pm_aslist_in_bp_asscalar)
print("pm_aslist_in_bp_as_list (unknown for now): %s" % pm_aslist_in_bp_as_list)
print("pm_asscalar_in_bp_asscalar: %s" % pm_asscalar_in_bp_asscalar)
print("pm_asscalar_in_bp_aslist (unknown for now): %s" %pm_asscalar_in_bp_aslist)
print("pm_total: %s" % (pm_aslist+pm_asscalar))
print("pm_sumextend: %s" % pm_sumextend)

BP
{'_id': {<class 'str'>: {'_stats': {'_count': 210030,
                                    '_max': 9,
                                    '_min': 1,
                                    '_sum': 1296281}}},
 '_stats': {'_count': 210030, '_max': 1, '_min': 1, '_sum': 210030},
 'go': {'BP': {'_stats': {'_count': 183388,
                          '_max': 1,
                          '_min': 1,
                          '_sum': 183388},
               'evidence': {<class 'str'>: {'_stats': {'_count': 77211,
                                                       '_max': 3,
                                                       '_min': 2,
                                                       '_sum': 196472}}},
               'id': {<class 'str'>: {'_stats': {'_count': 77211,
                                                 '_max': 10,
                                                 '_min': 10,
                                                 '_sum': 772110}}},
               'pubmed': {'_s

pm_aslist: 4067
pm_asscalar: 73280
pm_aslist_in_bp_asscalar: 279
pm_aslist_in_bp_as_list (unknown for now): 3788
pm_asscalar_in_bp_asscalar: 16673
pm_asscalar_in_bp_aslist (unknown for now): 56607
pm_total: 77347
pm_sumextend: 527523


In [72]:
import copy
import importlib
importlib.reload(inspect)
orig = copy.deepcopy(mapt)
print("orig")
pprint.pprint(orig["go"]["BP"][list]["pubmed"])
inspect.merge_scalar_list(mapt,"stats")
print("merged")
pprint.pprint(mapt["go"]["BP"][list]["pubmed"])
mapt = orig


orig
{<class 'list'>: {'_stats': {'_count': 20253,
                             '_max': 47,
                             '_min': 2,
                             '_sum': 50256},
                  <class 'int'>: {'_stats': {'_count': 2,
                                             '_max': 26404503,
                                             '_min': 25505245,
                                             '_sum': 51909748}}},
 <class 'int'>: {'_stats': {'_count': 249955,
                            '_max': 28325841,
                            '_min': 475,
                            '_sum': 4114372418731}}}
merged
{<class 'list'>: {'_stats': {'_count': 20532,
                             '_max': 47,
                             '_min': 2,
                             '_sum': 50839},
                  <class 'int'>: {'_stats': {'_count': 2,
                                             '_max': 26404503,
                                             '_min': 25505245,
                        

In [115]:
_id_count = mapt["_id"][str]["_stats"]["_count"]
go_BP_evidence = mapt["go"]["BP"]["evidence"][str]["_stats"]["_count"]
go_BP_list_evience = mapt["go"]["BP"][list]["evidence"][str]["_stats"]["_count"]
print("count(_id): %s" % _id_count)
print("go.BP.evidence: %s" % go_BP_evidence)
print("go_BP_list_evidence: %s" % go_BP_list_evidence)
print("go_BP_list_vs_scalar: %.2f%% as list, %.2f%% as scalar" % \
      (go_BP_evidence/(go_BP_evidence+go_BP_list_evidence)*100,
       go_BP_list_evidence/(go_BP_evidence+go_BP_list_evidence)*100))


count(_id): 210029
go.BP.evidence: 77227
go_BP_list_evidence: 755275
go_BP_list_vs_scalar: 9.28% as list, 90.72% as scalar


# Fields relations

In [67]:
#import importlib
#import biothings.utils.inspect as inspect
#importlib.reload(inspect)
maptm = inspect.inspect_docs(docs,mode="stats",merge=True)
pprint.pprint(maptm)

TypeError: 'int' object is not subscriptable

In [123]:
inspect.merge_scalar_list(maptm,mode="stats")
pprint.pprint(mapt)

{'_id': {<class 'str'>: {'_stats': {'_count': 210029,
                                    '_max': 9,
                                    '_min': 1,
                                    '_sum': 1296264}}},
 'go': {'BP': {'evidence': {<class 'str'>: {'_stats': {'_count': 77227,
                                                       '_max': 3,
                                                       '_min': 2,
                                                       '_sum': 196504}}},
               'id': {<class 'str'>: {'_stats': {'_count': 77227,
                                                 '_max': 10,
                                                 '_min': 10,
                                                 '_sum': 772270}}},
               'pubmed': {<class 'list'>: {'_stats': {'_count': 278,
                                                      '_max': 4,
                                                      '_min': 2,
                                                      '_sum': 5

In [81]:
rootks = mapt["go"].keys()
for rootk in rootks:
    elems = maptm["go"][rootk][list]
    elemscnt = elems["_stats"]["_count"]
    print("go.%s: %s docs (%.2f%% of total docs)" % \
          (rootk,elemscnt,elemscnt/_id_count*100))
    kkeys = maptm["go"][rootk][list].keys()
    for kkey in kkeys:
        try:
            kcount = elems[kkey][str]["_stats"]["_count"]
        except KeyError as e: # well, it's not generic (but it could), we know how GO is...
            print(elems[kkey].keys())
            kcount = elems[kkey][int]["_stats"]["_count"]
        print("    go.%s.%s: %s elements (%.2f%% of elements in the list)" % \
              (rootk,kkey,kcount,kcount/_id_count*100))

go.CC: 109441 docs (52.11% of total docs)
    go.CC.id: 554761 elements (264.14% of elements in the list)
    go.CC.term: 554761 elements (264.14% of elements in the list)
dict_keys([<class 'int'>, <class 'list'>])
    go.CC.pubmed: 119933 elements (57.10% of elements in the list)
    go.CC.evidence: 554761 elements (264.14% of elements in the list)
    go.CC.qualifier: 7780 elements (3.70% of elements in the list)
dict_keys(['_count', '_min', '_sum', '_max'])


KeyError: <class 'int'>